## Add SB1 geographic information.

In [55]:
import _utils
import numpy as np
import pandas as pd
from calitp.sql import to_snakecase

In [56]:
pd.options.display.max_columns = 200
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### Non SHOPP-ATP-TIRCP

In [57]:
# Read in 10 Year non SHOPP with ATP and TIRCP
df = to_snakecase(pd.read_excel(f"{_utils.GCS_FILE_PATH}cleaned_data_atp_tircp.xlsx"))

In [58]:
# Subset to join.
non_shopp_subset = [
    "ppno",
    "project_name",
    "lead_agency",
    "previous_caltrans_nominations",
    "full_county_name",
    "district",
    "project_description",
    "current_phase",
    "primary_mode",
    "urban_rural",
    "total_project_cost__$1,000",
    "total_unfunded_need__$1,000",
    "notes",
    "shs_capacity_increase_detail",
    "current_phase",
]

In [59]:
df = df[non_shopp_subset]

### SB1

In [60]:
# Read in SB1
sb1 = to_snakecase(pd.read_csv(f"{_utils.GCS_FILE_PATH}RebuildingCA_map_Data.csv"))

In [70]:
sb1.sample()

,project_id,project_name,implementing_agency,sb1_program,federal_program,fiscal_year,project_description,total_cost,sb1_funds,is_sb1?,federal_funds,is_federal?,project_status,assembly_districts,senate_districts,congressional_districts,counties,cities,caltrans_districts,on_shs?,date_updated
4098,'0418000347,Roadside,Caltrans,State Hwy Operations & Protection Program Minor A,NaN,'18/19,"In Napa, on Routes 121 and 128. Replace trees after fire.",778700.0,0.0,N,NaN,NaN,In Progress,04,03,05,Napa,NaN,04,Y,10/08/2021


In [61]:
sb1_subset = [
    "project_name",
    "implementing_agency",
    "sb1_program",
    "fiscal_year",
    "project_description",
    "total_cost",
    "sb1_funds",
    "is_sb1?",
    "project_status",
    "assembly_districts",
    "senate_districts",
    "congressional_districts",
    "counties",
    "cities",
    "caltrans_districts",
    "on_shs?",
]

In [62]:
sb1.sb1_program.value_counts()

Local Streets and Roads                              3345
State Hwy Operations & Protection Program Major      2771
Highway Maintenance                                  1002
Active Transportation Program                         923
State of Good Repair                                  536
State Transit Assistance                              196
State Transportation Improvement Program              188
State Hwy Operations & Protection Program Minor A     169
Local Partnership Program - Formulaic                 135
Transit and Intercity Rail Capital Program             96
Trade Corridor Enhancement Program                     66
Local Partnership Program - Competitive                62
State Rail Assistance                                  53
Field Maintenance                                      48
Solutions for Congested Corridors Program              42
Name: sb1_program, dtype: int64

In [63]:
sb1_2 = sb1[sb1_subset]

In [64]:
# Filter out completed projects
sb1.project_status.value_counts()

Completed      4838
Planned        2517
In Progress    2277
Name: project_status, dtype: int64

In [65]:
sb1_2 = sb1_2.loc[sb1_2.project_status != "Completed"].reset_index(drop=True)

In [66]:
# Fill in NA
sb1_2 = sb1_2.fillna(sb1_2.dtypes.replace({"float64": 0.0, "object": "None"}))

In [67]:
# Align funding
sb1_2 = _utils.align_funding_numbers(sb1_2, ["total_cost", "sb1_funds",])

In [68]:
df.sample()

,ppno,project_name,lead_agency,previous_caltrans_nominations,full_county_name,district,project_description,current_phase,primary_mode,urban_rural,"total_project_cost__$1,000","total_unfunded_need__$1,000",notes,shs_capacity_increase_detail,current_phase
345,None,La River Bikeway - Segment 8,La Metro,None,Los Angeles,7,"This project will construct at least 4.65 miles of high-quality bicycle infrastructure and pedestrian enhancements along the LA River and through the east San Fernando Valley, providing a safe and continuous active transportation network that will expand non-auto commute options increase access to high-quality transit lines and employment hubs, and improve access to recreation for LA residents and visitors. This project will include 3.2 miles of Class I bike, path greenway, and pedestrian path (as feasible), along the LA river between Whitsett Ave to Lankershim Blvd, which will also include five (5) at grade crossings at, and four (4) at grade crossings located at Whitsett Ave, Laurel Canyon Blvd, and Tujunga Ave, as well as two (2) bridges located at Radford Ave/over the Tujunga Wash as and a pedestrian bridge over the LA River. Class IV bike path on Whitsett, Radford, and Vineland will be constructed and approximately 0.45 miles of Class III bike path on Laurel Grove leading to the River will be constructed. This project will also try to upgrade to the maximum extent feasible the Vineland Ave corridor from Whipple to San Fernando Road, which stretches approximately 5 miles. These upgrades include putting in bollards along buffer areas and painting conflict zones. Other scope includes lighting, bioswales, retaining walls (as needed), and other pedestrian amenities, such as bike racks, benches, trash cans, and pet waste stations, and pedestrian enhancements.",UNKNOWN,Bike/Pedestrian,Urban,34108.0,23409.0,Partner agency = Implementing Agency,None,UNKNOWN


In [69]:
sb1_2.sample()

,project_name,implementing_agency,sb1_program,fiscal_year,project_description,total_cost,sb1_funds,is_sb1?,project_status,assembly_districts,senate_districts,congressional_districts,counties,cities,caltrans_districts,on_shs?
922,Drainage - HM251,Caltrans,Highway Maintenance,'21/22,Repair/Replace Culverts,333.0,333.0,Y,In Progress,28,"13, 15",18,Santa Clara,Saratoga,04,N
